In [1]:
import sys
import os
import csv
import math

import scipy.stats as stats
import pandas as pd
import urllib
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
STI_WIDTH = 1920
STI_HEIGHT = 1080

fDirPath = "./gaze_features"
fileList = os.listdir(fDirPath)
F_IDX = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
F_NAME = ["center-bias", "contrast_color", "contrast_intensity", "contrast_orientation", "HOG", "horizontal_line", "LOG_spectrum", "saliency_color", "saliency_intensity", "saliency_orientation", "computed_saliency"]

featureData = []
groupData = []
groupIdx = 1
fileCount = 0

In [3]:
for _fIdx in F_IDX:
    groupIdx = 1
    groupData = []
    
    # read feature values
    for _fileName in fileList:
        filePath = fDirPath + "/" + _fileName

        rf = open(filePath, 'r', encoding='utf-8')
        rdr = csv.reader(rf)
        
        _t = True
        for _row in rdr:
            if _t:
                _t = False
                continue
            # out of range error exception
            _x = int(math.trunc(float(_row[1])))
            _y = int(math.trunc(float(_row[2])))
            if _x >= STI_WIDTH or _y >= STI_HEIGHT or _x < 0 or _y < 0:
                continue
            groupData.append([float(_row[_fIdx]), groupIdx])
        rf.close()
        groupIdx += 1
        
        if _fIdx == 2:
            fileCount += 1
    featureData.append(groupData)
print(fileCount)
print("loaded all feature data from %d number of .csv files"%fileCount)

60
loaded all feature data from 60 number of .csv files


In [4]:
import warnings
idx = 0

for i in range(0, len(F_IDX)):    
    print("___________________")
    print("feature: "+F_NAME[i])
    print("F_IDX idx = %d"%i)
    print(featureData[0][0])
    
    _gnp = np.empty((0,2), float)
    for _row in featureData[i]:
        _gnp = np.append(_gnp, np.array([[_row[0], _row[1]]]), axis=0)
    print(_gnp)   
    #_gnp = np.array(featureData[i])
    #print(featureData[i])
    # F_statistic, pVal = stats.f_oneway(group1, group2, group3, group4, group5, group6, group7, group8, group9, group10)
    # print('일원분산분석 결과 : F={0:.1f}, p={1:.5f}'.format(F_statistic, pVal))
    # if pVal < 0.05:
    #     print('P-value 값이 충분히 작음으로 인해 그룹의 평균값이 통계적으로 유의미하게 차이납니다.')

    # 경고 메세지 무시하기
    warnings.filterwarnings('ignore')

    df = pd.DataFrame(_gnp, columns=['value', 'stimulus'])

    # the "C" indicates categorical data
    model = ols('value ~ C(stimulus)', df).fit()

    print(anova_lm(model))
    print(model.summary())

___________________
feature: center-bias
F_IDX idx = 0
[471.64165019989, 1]
[[471.6416502    1.        ]
 [338.4852612    1.        ]
 [344.77485895   1.        ]
 ...
 [584.13497686  60.        ]
 [600.2016449   60.        ]
 [587.74405003  60.        ]]
                  df        sum_sq       mean_sq          F  PR(>F)
C(stimulus)     59.0  1.247285e+08  2.114043e+06  65.476068     0.0
Residual     11276.0  3.640712e+08  3.228727e+04        NaN     NaN
                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.251
Method:                 Least Squares   F-statistic:                     65.48
Date:                Wed, 26 Aug 2020   Prob (F-statistic):               0.00
Time:                        13:06:35   Log-Likelihood:                -74903.
No. Observations:               11336   AIC:                      

In [5]:
featureCatData = []
gIdx = 1

for _fIdx in F_IDX:
    
    # read feature values
    for _fileName in fileList:
        filePath = fDirPath + "/" + _fileName

        rf = open(filePath, 'r', encoding='utf-8')
        rdr = csv.reader(rf)
        
        _t = True
        for _row in rdr:
            if _t:
                _t = False
                continue
            # out of range error exception
            _x = int(math.trunc(float(_row[1])))
            _y = int(math.trunc(float(_row[2])))
            if _x >= STI_WIDTH or _y >= STI_HEIGHT or _x < 0 or _y < 0:
                continue
            featureCatData.append([float(_row[_fIdx]), gIdx])
        rf.close()
    gIdx += 1


In [6]:
_fnp = np.empty((0,2), float)
for _row in featureCatData:
    _fnp = np.append(_fnp, np.array([[_row[0], _row[1]]]), axis=0)
print(_fnp)

[[4.71641650e+02 1.00000000e+00]
 [3.38485261e+02 1.00000000e+00]
 [3.44774859e+02 1.00000000e+00]
 ...
 [1.70359620e-01 1.10000000e+01]
 [1.79452650e-01 1.10000000e+01]
 [1.78968160e-01 1.10000000e+01]]


In [7]:
fdf = pd.DataFrame(_fnp, columns=['value', 'feature'])

# the "C" indicates categorical data
fmodel = ols('value ~ C(feature)', fdf).fit()

print(anova_lm(fmodel))
print(fmodel.summary())

                  df        sum_sq       mean_sq             F  PR(>F)
C(feature)      10.0  2.935686e+09  2.935686e+08  68228.051202     0.0
Residual    124685.0  5.364890e+08  4.302755e+03           NaN     NaN
                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                 6.823e+04
Date:                Wed, 26 Aug 2020   Prob (F-statistic):               0.00
Time:                        13:07:51   Log-Likelihood:            -6.9860e+05
No. Observations:              124696   AIC:                         1.397e+06
Df Residuals:                  124685   BIC:                         1.397e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                       